In [ ]:
import quest
import panel as pn
import holoviews as hv
import geoviews as gv
import cartopy.crs as ccrs

from holoviews.operation.datashader import rasterize

hv.extension('bokeh')

## Select tiles to download

In [ ]:
quest_service = 'svc://wmts:seamless_imagery'
tile_service_options = quest.api.get_download_options(quest_service, fmt='param')[quest_service]
tile_service_options.params()['bbox'].precedence = -1  # hide bbox input
pn.panel(tile_service_options.param)

In [ ]:
tiles = gv.WMTS(tile_service_options.url).options(width=950, height=600)
boxes = gv.Polygons(
    hv.Bounds((-72.43925984610391, 45.8471360126193, -68.81252476472281, 47.856449699679516))
).options(fill_alpha=0.4, color='blue', line_color='blue', line_width=2)
box_stream = hv.streams.BoxEdit(source=boxes, num_objects=1)
tiles * boxes

## Define query and download

In [ ]:
data = box_stream.data
bbox = [data['x0'][0], data['y0'][0], data['x1'][0], data['y1'][0]]
tile_service_options.bbox = bbox

In [ ]:
dataset_metadata = quest.api.get_data(
    service_uri=quest_service,
    search_filters=None,
    download_options=tile_service_options,
    collection_name='examples',
    use_cache=True,
    as_open_datasets=False,
    expand=True,
)[0]
file_path = dataset_metadata['file_path']

## Open and display downloaded tiles

In [ ]:
rgb = gv.RGB.load_tiff(file_path, crs=ccrs.GOOGLE_MERCATOR)
gv.tile_sources.Wikipedia * rasterize(rgb).options(width=800, height=600, active_tools=['wheel_zoom'])